# Supervised Models for Fraud Detection

**Notebook:** 04_supervised_models.ipynb  
**Objective:** Develop and evaluate baseline XGBoost model with iterative feature addition

## Modeling Strategy

**Baseline Approach:**
- Start with Critical Priority features only (transaction_count_bin, card_age_bin, hour)
- Use XGBoost with class weights to handle imbalance
- Evaluate using Precision-Recall Curve, F1-Score, and AUC-PR

**Iterative Feature Addition:**
- Add features based on importance rankings
- Monitor performance improvements
- Stop when diminishing returns are observed

In [ ]:
# ============================================================
# GLOBAL IMPORTS & DEPENDENCIES
# ============================================================

import os
import sys
from pathlib import Path
from typing import Tuple, List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Data Processing
import pandas as pd
import numpy as np

# Model Evaluation
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_recall_curve,
    roc_curve,
    roc_auc_score,
    average_precision_score,
    f1_score,
    precision_score,
    recall_score
)

# XGBoost
import xgboost as xgb

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
import pickle
import joblib
from datetime import datetime

print("All dependencies loaded successfully")

In [ ]:
# ============================================================
# CONFIGURATION & PATHS
# ============================================================

# Base paths
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR = PROJECT_ROOT / 'data'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
MODELS_DIR = PROJECT_ROOT / 'models'
RESULTS_DIR = PROJECT_ROOT / 'results'

# Create directories if they don't exist
MODELS_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

# Preprocessed data paths
PREPROCESSED_TRAIN_PATH = PROCESSED_DATA_DIR / 'train_preprocessed.parquet'
PREPROCESSED_VAL_PATH = PROCESSED_DATA_DIR / 'val_preprocessed.parquet'
PREPROCESSED_TEST_PATH = PROCESSED_DATA_DIR / 'test_preprocessed.parquet'
PREPROCESSER_PATH = MODELS_DIR / 'preprocessor.pkl'
FEATURE_NAMES_PATH = MODELS_DIR / 'feature_names.pkl'

print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Models directory: {MODELS_DIR}")
print(f"Results directory: {RESULTS_DIR}")

## 1. Load Preprocessed Data

In [ ]:
# ============================================================
# LOAD PREPROCESSED DATA
# ============================================================

def load_preprocessed_data() -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Load preprocessed train, validation, and test datasets.
    
    Returns:
        Tuple of (train_df, val_df, test_df)
    """
    if not PREPROCESSED_TRAIN_PATH.exists():
        raise FileNotFoundError(
            f"Preprocessed data not found: {PREPROCESSED_TRAIN_PATH}\n"
            "Please run the preprocessing notebook (02_preprocessing.ipynb) first."
        )
    
    print("Loading preprocessed data...")
    train_df = pd.read_parquet(PREPROCESSED_TRAIN_PATH)
    val_df = pd.read_parquet(PREPROCESSED_VAL_PATH)
    test_df = pd.read_parquet(PREPROCESSED_TEST_PATH)
    
    print(f"\n✓ Data loaded successfully")
    print(f"  Train: {train_df.shape[0]:,} samples, {train_df.shape[1]} features")
    print(f"  Validation: {val_df.shape[0]:,} samples, {val_df.shape[1]} features")
    print(f"  Test: {test_df.shape[0]:,} samples, {test_df.shape[1]} features")
    
    print(f"\nFraud rates:")
    print(f"  Train: {train_df['is_fraud'].mean():.4%}")
    print(f"  Validation: {val_df['is_fraud'].mean():.4%}")
    print(f"  Test: {test_df['is_fraud'].mean():.4%}")
    
    return train_df, val_df, test_df

# Load data
train_df, val_df, test_df = load_preprocessed_data()

# Separate features and target
feature_cols = [col for col in train_df.columns if col != 'is_fraud']
print(f"\nAvailable features: {len(feature_cols)}")
print(f"Feature names: {feature_cols}")

## 2. Define Feature Sets

In [ ]:
# ============================================================
# FEATURE SET DEFINITIONS
# ============================================================

# Critical Priority features (from EDA findings)
CRITICAL_FEATURES = [
    'transaction_count_bin',
    'card_age_bin',
    'hour',
    'time_bin',
    'is_peak_fraud_hour',
    'is_new_card',
    'is_low_volume_card'
]

# High Priority features
HIGH_PRIORITY_FEATURES = [
    'category',
    'day_of_week',
    'month',
    'is_peak_fraud_day',
    'is_peak_fraud_season',
    'is_high_risk_category',
    'card_age_days',
    'transaction_count'
]

# Interaction features
INTERACTION_FEATURES = [
    'evening_high_amount',
    'evening_online_shopping',
    'large_city_evening',
    'new_card_evening',
    'high_amount_online'
]

# Enriched features
ENRICHED_FEATURES = [
    'temporal_risk_score',
    'geographic_risk_score',
    'card_risk_score',
    'risk_tier'
]

# Filter to only features that exist in the dataset
def filter_available_features(feature_list: List[str], available_cols: List[str]) -> List[str]:
    """Filter feature list to only include features that exist in the dataset."""
    return [f for f in feature_list if f in available_cols]

CRITICAL_FEATURES_AVAIL = filter_available_features(CRITICAL_FEATURES, feature_cols)
HIGH_PRIORITY_FEATURES_AVAIL = filter_available_features(HIGH_PRIORITY_FEATURES, feature_cols)
INTERACTION_FEATURES_AVAIL = filter_available_features(INTERACTION_FEATURES, feature_cols)
ENRICHED_FEATURES_AVAIL = filter_available_features(ENRICHED_FEATURES, feature_cols)

print("Feature Set Summary:")
print(f"  Critical Priority: {len(CRITICAL_FEATURES_AVAIL)}/{len(CRITICAL_FEATURES)} available")
print(f"  High Priority: {len(HIGH_PRIORITY_FEATURES_AVAIL)}/{len(HIGH_PRIORITY_FEATURES)} available")
print(f"  Interaction: {len(INTERACTION_FEATURES_AVAIL)}/{len(INTERACTION_FEATURES)} available")
print(f"  Enriched: {len(ENRICHED_FEATURES_AVAIL)}/{len(ENRICHED_FEATURES)} available")

if CRITICAL_FEATURES_AVAIL:
    print(f"\nCritical features to use: {CRITICAL_FEATURES_AVAIL}")
else:
    print("\n⚠ Warning: No critical features found in dataset!")

## 3. Evaluation Metrics Functions

In [ ]:
# ============================================================
# EVALUATION METRICS FUNCTIONS
# ============================================================

def evaluate_model(
    model: xgb.XGBClassifier,
    X: np.ndarray,
    y: np.ndarray,
    dataset_name: str = "Dataset"
) -> Dict[str, float]:
    """
    Evaluate model performance on a dataset.
    
    Args:
        model: Trained XGBoost model
        X: Feature matrix
        y: True labels
        dataset_name: Name of dataset for display
    
    Returns:
        Dictionary of evaluation metrics
    """
    # Predictions
    y_pred = model.predict(X)
    y_pred_proba = model.predict_proba(X)[:, 1]
    
    # Calculate metrics
    metrics = {
        'f1_score': f1_score(y, y_pred),
        'precision': precision_score(y, y_pred),
        'recall': recall_score(y, y_pred),
        'roc_auc': roc_auc_score(y, y_pred_proba),
        'pr_auc': average_precision_score(y, y_pred_proba)
    }
    
    # Print results
    print(f"\n{dataset_name} Performance:")
    print(f"  F1-Score: {metrics['f1_score']:.4f}")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall: {metrics['recall']:.4f}")
    print(f"  ROC-AUC: {metrics['roc_auc']:.4f}")
    print(f"  PR-AUC: {metrics['pr_auc']:.4f}")
    
    # Confusion matrix
    cm = confusion_matrix(y, y_pred)
    print(f"\n  Confusion Matrix:")
    print(f"    TN: {cm[0,0]:,}  FP: {cm[0,1]:,}")
    print(f"    FN: {cm[1,0]:,}  TP: {cm[1,1]:,}")
    print(f"    False Positive Rate: {cm[0,1]/(cm[0,0]+cm[0,1]):.4%}")
    
    return metrics

def plot_precision_recall_curve(
    y_true: np.ndarray,
    y_pred_proba: np.ndarray,
    dataset_name: str = "Dataset",
    ax: Optional[plt.Axes] = None
) -> None:
    """Plot Precision-Recall curve."""
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred_proba)
    pr_auc = average_precision_score(y_true, y_pred_proba)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))
    
    ax.plot(recall, precision, label=f'{dataset_name} (AUC={pr_auc:.4f})')
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_title('Precision-Recall Curve')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    return ax

def plot_roc_curve(
    y_true: np.ndarray,
    y_pred_proba: np.ndarray,
    dataset_name: str = "Dataset",
    ax: Optional[plt.Axes] = None
) -> None:
    """Plot ROC curve."""
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))
    
    ax.plot(fpr, tpr, label=f'{dataset_name} (AUC={roc_auc:.4f})')
    ax.plot([0, 1], [0, 1], 'k--', label='Random')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('ROC Curve')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    return ax

print("Evaluation functions defined")

## 4. Baseline Model: XGBoost with Critical Priority Features

In [ ]:
# ============================================================
# BASELINE MODEL: XGBOOST WITH CRITICAL FEATURES ONLY
# ============================================================

# Prepare data with Critical Priority features only
baseline_features = CRITICAL_FEATURES_AVAIL if CRITICAL_FEATURES_AVAIL else feature_cols[:7]

X_train_baseline = train_df[baseline_features].values
y_train_baseline = train_df['is_fraud'].values

X_val_baseline = val_df[baseline_features].values
y_val_baseline = val_df['is_fraud'].values

X_test_baseline = test_df[baseline_features].values
y_test_baseline = test_df['is_fraud'].values

print(f"Baseline model using {len(baseline_features)} features:")
print(f"  Features: {baseline_features}")
print(f"\nData shapes:")
print(f"  Train: X={X_train_baseline.shape}, y={y_train_baseline.shape}")
print(f"  Validation: X={X_val_baseline.shape}, y={y_val_baseline.shape}")
print(f"  Test: X={X_test_baseline.shape}, y={y_test_baseline.shape}")

In [ ]:
# ============================================================
# TRAIN BASELINE XGBOOST MODEL WITH CLASS WEIGHTS
# ============================================================

# Calculate class weights (inverse of class frequency)
n_samples = len(y_train_baseline)
n_fraud = y_train_baseline.sum()
n_legitimate = n_samples - n_fraud

# XGBoost uses scale_pos_weight parameter
# scale_pos_weight = (number of negative samples) / (number of positive samples)
scale_pos_weight = n_legitimate / n_fraud

print(f"Class distribution:")
print(f"  Legitimate: {n_legitimate:,} ({n_legitimate/n_samples:.2%})")
print(f"  Fraud: {n_fraud:,} ({n_fraud/n_samples:.2%})")
print(f"  scale_pos_weight: {scale_pos_weight:.2f}")

# Train baseline model
baseline_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)

print("\nTraining baseline model...")
baseline_model.fit(
    X_train_baseline,
    y_train_baseline,
    eval_set=[(X_val_baseline, y_val_baseline)],
    verbose=False
)

print("✓ Baseline model trained")

In [ ]:
# ============================================================
# EVALUATE BASELINE MODEL
# ============================================================

baseline_train_metrics = evaluate_model(baseline_model, X_train_baseline, y_train_baseline, "Train")
baseline_val_metrics = evaluate_model(baseline_model, X_val_baseline, y_val_baseline, "Validation")
baseline_test_metrics = evaluate_model(baseline_model, X_test_baseline, y_test_baseline, "Test")

In [ ]:
# ============================================================
# VISUALIZE BASELINE MODEL PERFORMANCE
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Precision-Recall curves
y_train_proba = baseline_model.predict_proba(X_train_baseline)[:, 1]
y_val_proba = baseline_model.predict_proba(X_val_baseline)[:, 1]
y_test_proba = baseline_model.predict_proba(X_test_baseline)[:, 1]

plot_precision_recall_curve(y_train_baseline, y_train_proba, "Train", axes[0])
plot_precision_recall_curve(y_val_baseline, y_val_proba, "Validation", axes[0])
plot_precision_recall_curve(y_test_baseline, y_test_proba, "Test", axes[0])

# ROC curves
plot_roc_curve(y_train_baseline, y_train_proba, "Train", axes[1])
plot_roc_curve(y_val_baseline, y_val_proba, "Validation", axes[1])
plot_roc_curve(y_test_baseline, y_test_proba, "Test", axes[1])

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# FEATURE IMPORTANCE ANALYSIS (BASELINE)
# ============================================================

feature_importance = baseline_model.feature_importances_
importance_df = pd.DataFrame({
    'feature': baseline_features,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("Baseline Model Feature Importance:")
print(importance_df.to_string(index=False))

# Visualize feature importance
plt.figure(figsize=(10, 6))
sns.barplot(data=importance_df, x='importance', y='feature', palette='viridis')
plt.title('Baseline Model - Feature Importance')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

# Store baseline results
baseline_results = {
    'features': baseline_features,
    'train_metrics': baseline_train_metrics,
    'val_metrics': baseline_val_metrics,
    'test_metrics': baseline_test_metrics,
    'feature_importance': importance_df.to_dict('records')
}

print("\n✓ Baseline model evaluation complete")

## 5. Iterative Feature Addition

In [ ]:
# ============================================================
# ITERATIVE FEATURE ADDITION FUNCTION
# ============================================================

def train_and_evaluate_model(
    features: List[str],
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    test_df: pd.DataFrame,
    scale_pos_weight: float
) -> Tuple[xgb.XGBClassifier, Dict[str, Dict[str, float]], pd.DataFrame]:
    """
    Train XGBoost model with given features and return metrics.
    
    Returns:
        Tuple of (model, metrics_dict, feature_importance_df)
    """
    # Prepare data
    X_train = train_df[features].values
    y_train = train_df['is_fraud'].values
    X_val = val_df[features].values
    y_val = val_df['is_fraud'].values
    X_test = test_df[features].values
    y_test = test_df['is_fraud'].values
    
    # Train model
    model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        eval_metric='logloss',
        use_label_encoder=False
    )
    
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    
    # Evaluate
    train_metrics = evaluate_model(model, X_train, y_train, "Train")
    val_metrics = evaluate_model(model, X_val, y_val, "Validation")
    test_metrics = evaluate_model(model, X_test, y_test, "Test")
    
    metrics = {
        'train': train_metrics,
        'val': val_metrics,
        'test': test_metrics
    }
    
    # Feature importance
    importance_df = pd.DataFrame({
        'feature': features,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    return model, metrics, importance_df

print("Iterative feature addition function defined")

In [ ]:
# ============================================================
# ITERATIVE FEATURE ADDITION: BUILD FEATURE SETS
# ============================================================

# Define feature addition order based on priority
feature_sets = []

# Step 1: Critical features only (baseline)
feature_sets.append({
    'name': 'Critical Only',
    'features': CRITICAL_FEATURES_AVAIL
})

# Step 2: Critical + High Priority
if HIGH_PRIORITY_FEATURES_AVAIL:
    feature_sets.append({
        'name': 'Critical + High Priority',
        'features': CRITICAL_FEATURES_AVAIL + HIGH_PRIORITY_FEATURES_AVAIL
    })

# Step 3: Critical + High Priority + Interactions
if INTERACTION_FEATURES_AVAIL:
    feature_sets.append({
        'name': 'Critical + High Priority + Interactions',
        'features': CRITICAL_FEATURES_AVAIL + HIGH_PRIORITY_FEATURES_AVAIL + INTERACTION_FEATURES_AVAIL
    })

# Step 4: All features
if ENRICHED_FEATURES_AVAIL:
    feature_sets.append({
        'name': 'All Features',
        'features': CRITICAL_FEATURES_AVAIL + HIGH_PRIORITY_FEATURES_AVAIL + INTERACTION_FEATURES_AVAIL + ENRICHED_FEATURES_AVAIL
    })

print(f"Feature addition sequence ({len(feature_sets)} steps):")
for i, fs in enumerate(feature_sets, 1):
    print(f"  Step {i}: {fs['name']} ({len(fs['features'])} features)")

In [ ]:
# ============================================================
# ITERATIVE FEATURE ADDITION: TRAIN MODELS
# ============================================================

all_results = []
all_models = []

print("=" * 80)
print("ITERATIVE FEATURE ADDITION")
print("=" * 80)

for i, feature_set in enumerate(feature_sets):
    print(f"\n{'='*80}")
    print(f"Step {i+1}/{len(feature_sets)}: {feature_set['name']}")
    print(f"{'='*80}")
    print(f"Features ({len(feature_set['features'])}): {feature_set['features']}")
    
    # Train and evaluate
    model, metrics, importance_df = train_and_evaluate_model(
        feature_set['features'],
        train_df,
        val_df,
        test_df,
        scale_pos_weight
    )
    
    # Store results
    result = {
        'step': i + 1,
        'name': feature_set['name'],
        'n_features': len(feature_set['features']),
        'features': feature_set['features'],
        'metrics': metrics,
        'feature_importance': importance_df,
        'model': model
    }
    
    all_results.append(result)
    all_models.append(model)
    
    # Print improvement over previous step
    if i > 0:
        prev_val_f1 = all_results[i-1]['metrics']['val']['f1_score']
        curr_val_f1 = metrics['val']['f1_score']
        improvement = curr_val_f1 - prev_val_f1
        print(f"\n  Improvement in Validation F1: {improvement:+.4f} ({improvement/prev_val_f1*100:+.2f}%)")

print(f"\n{'='*80}")
print("ITERATIVE FEATURE ADDITION COMPLETE")
print(f"{'='*80}")

In [ ]:
# ============================================================
# COMPARE MODEL PERFORMANCE ACROSS FEATURE SETS
# ============================================================

# Create comparison DataFrame
comparison_data = []
for result in all_results:
    comparison_data.append({
        'Feature Set': result['name'],
        'N Features': result['n_features'],
        'Train F1': result['metrics']['train']['f1_score'],
        'Val F1': result['metrics']['val']['f1_score'],
        'Test F1': result['metrics']['test']['f1_score'],
        'Val PR-AUC': result['metrics']['val']['pr_auc'],
        'Test PR-AUC': result['metrics']['test']['pr_auc'],
        'Val ROC-AUC': result['metrics']['val']['roc_auc'],
        'Test ROC-AUC': result['metrics']['test']['roc_auc']
    })

comparison_df = pd.DataFrame(comparison_data)
print("\nModel Performance Comparison:")
print(comparison_df.to_string(index=False))

# Visualize performance comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# F1-Score comparison
axes[0, 0].plot(comparison_df['N Features'], comparison_df['Train F1'], 'o-', label='Train', linewidth=2)
axes[0, 0].plot(comparison_df['N Features'], comparison_df['Val F1'], 's-', label='Validation', linewidth=2)
axes[0, 0].plot(comparison_df['N Features'], comparison_df['Test F1'], '^-', label='Test', linewidth=2)
axes[0, 0].set_xlabel('Number of Features')
axes[0, 0].set_ylabel('F1-Score')
axes[0, 0].set_title('F1-Score vs Number of Features')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# PR-AUC comparison
axes[0, 1].plot(comparison_df['N Features'], comparison_df['Val PR-AUC'], 's-', label='Validation', linewidth=2)
axes[0, 1].plot(comparison_df['N Features'], comparison_df['Test PR-AUC'], '^-', label='Test', linewidth=2)
axes[0, 1].set_xlabel('Number of Features')
axes[0, 1].set_ylabel('PR-AUC')
axes[0, 1].set_title('Precision-Recall AUC vs Number of Features')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# ROC-AUC comparison
axes[1, 0].plot(comparison_df['N Features'], comparison_df['Val ROC-AUC'], 's-', label='Validation', linewidth=2)
axes[1, 0].plot(comparison_df['N Features'], comparison_df['Test ROC-AUC'], '^-', label='Test', linewidth=2)
axes[1, 0].set_xlabel('Number of Features')
axes[1, 0].set_ylabel('ROC-AUC')
axes[1, 0].set_title('ROC-AUC vs Number of Features')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Feature importance for best model (highest validation F1)
best_idx = comparison_df['Val F1'].idxmax()
best_result = all_results[best_idx]
importance_df_best = best_result['feature_importance'].head(15)

axes[1, 1].barh(range(len(importance_df_best)), importance_df_best['importance'].values)
axes[1, 1].set_yticks(range(len(importance_df_best)))
axes[1, 1].set_yticklabels(importance_df_best['feature'].values)
axes[1, 1].set_xlabel('Importance')
axes[1, 1].set_title(f'Top 15 Features - {best_result["name"]}')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# IDENTIFY BEST MODEL
# ============================================================

# Find best model based on validation F1-score
best_idx = comparison_df['Val F1'].idxmax()
best_result = all_results[best_idx]
best_model = best_result['model']

print("=" * 80)
print("BEST MODEL SELECTION")
print("=" * 80)
print(f"\nBest Model: {best_result['name']}")
print(f"  Number of Features: {best_result['n_features']}")
print(f"\nValidation Performance:")
print(f"  F1-Score: {best_result['metrics']['val']['f1_score']:.4f}")
print(f"  PR-AUC: {best_result['metrics']['val']['pr_auc']:.4f}")
print(f"  ROC-AUC: {best_result['metrics']['val']['roc_auc']:.4f}")
print(f"\nTest Performance:")
print(f"  F1-Score: {best_result['metrics']['test']['f1_score']:.4f}")
print(f"  PR-AUC: {best_result['metrics']['test']['pr_auc']:.4f}")
print(f"  ROC-AUC: {best_result['metrics']['test']['roc_auc']:.4f}")

print(f"\nTop 10 Most Important Features:")
print(best_result['feature_importance'].head(10).to_string(index=False))

## 6. Save Best Model

In [ ]:
# ============================================================
# SAVE BEST MODEL AND RESULTS
# ============================================================

# Save best model
best_model_path = MODELS_DIR / 'xgb_best_model.pkl'
joblib.dump(best_model, best_model_path)
print(f"Best model saved: {best_model_path}")

# Save model metadata
model_metadata = {
    'model_name': 'XGBoost',
    'feature_set_name': best_result['name'],
    'features': best_result['features'],
    'n_features': best_result['n_features'],
    'metrics': best_result['metrics'],
    'feature_importance': best_result['feature_importance'].to_dict('records'),
    'scale_pos_weight': scale_pos_weight,
    'training_date': datetime.now().isoformat()
}

metadata_path = MODELS_DIR / 'xgb_best_model_metadata.pkl'
joblib.dump(model_metadata, metadata_path)
print(f"Model metadata saved: {metadata_path}")

# Save all results for comparison
all_results_path = RESULTS_DIR / 'iterative_feature_addition_results.pkl'
joblib.dump(all_results, all_results_path)
print(f"All results saved: {all_results_path}")

# Save comparison DataFrame
comparison_df_path = RESULTS_DIR / 'model_comparison.csv'
comparison_df.to_csv(comparison_df_path, index=False)
print(f"Comparison DataFrame saved: {comparison_df_path}")

print("\n✓ Model saving complete!")